In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import os

import numpy as np

df=pd.read_csv('./data/train.csv')

In [2]:
def limpia_catergoricas(data):

    def limpia_color(df):
        return df.apply(lambda x: abs(ord(x.upper())-ord('Z')))


    def limpia_corte(df):
        dict_cut={'Ideal':5,'Premium':4,  'Very Good':3,'Good':2,  'Fair':1, 'Poor':0}
        return df.map(dict_cut)

    def limpia_claridad(df):
        dict_clarity={'I3':0, 'I2':1, 'I1':2, 'SI2':3, 'SI1':4, 'VS2':5, 
                    'VS1':6, 'VVS2':7, 'VVS1':8, 'IF':9,'F':10}
        return df.map(dict_clarity)

    data.color=limpia_color(data.color)
    data.cut=limpia_corte(data.cut)
    data.clarity=limpia_claridad(data.clarity)
    return data
    

def limpia_dimensiones(data):
    data.columns=['id', 'carat', 'cut', 'color', 'clarity', 'depth_p', 'table_p', 'x', 'y',
       'z', 'price']
    return data[['cut', 'color', 'clarity', 'carat', 'depth_p', 'table_p','price']]

def incluye_ideal_table(data,ideal):
    data['ideal_table']=abs(data.table_p-ideal)
    return data.drop('table_p',axis=1)


In [5]:
df=limpia_catergoricas(df)
df=limpia_dimensiones(df)

df_size = df[df.carat - np.floor(df.carat) == 0]
df_off = df[df.carat - np.floor(df.carat) != 0]
#df=df_size
#df=df_off
df

,cut,color,clarity,carat,depth_p,table_p,price
0,4,19,6,1.01,62.7,58.0,8.754
1,2,19,5,0.60,64.3,57.0,7.492
2,4,18,3,1.60,59.4,59.0,9.010
3,3,16,6,0.30,63.5,58.0,6.227
4,3,20,3,1.00,62.8,57.0,8.314
...,...,...,...,...,...,...,...
40450,5,21,5,0.51,62.5,55.0,7.412
40451,5,18,5,0.80,62.1,54.0,8.016
40452,4,19,3,1.53,58.7,59.0,9.070
40453,5,17,5,1.51,61.6,54.0,9.199


In [6]:
import h2o
from h2o.estimators import H2ORandomForestEstimator



In [7]:

h2o.init(ip="localhost", port=54323)

Checking whether there is an H2O instance running at http://localhost:54323 . connected.


H2O_cluster_uptime:,4 mins 43 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,11 months and 7 days !!!
H2O_cluster_name:,H2O_from_python_emilio_6b7muq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,0
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [9]:

dia=h2o.H2OFrame(df)


# Set the predictors and response;
# set the response as a factor:

predictors = ['cut','color','clarity','carat','depth_p','table_p']
response = 'price'

# Split the dataset into a train and valid set:
train, valid = dia.split_frame(ratios=[.8])

H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Total input file size of 1.3 MB is much larger than total cluster memory of Zero  , please use either a larger cluster or smaller data.
  Request: POST /3/Parse
    data: {'destination_frame': 'Key_Frame__upload_aaba0099ca02d395d0dcc4d4a36849c6.hex', 'parse_type': 'CSV', 'separator': '44', 'single_quotes': 'False', 'check_header': '1', 'number_columns': '7', 'chunk_size': '4194304', 'delete_on_done': 'True', 'blocking': 'False', 'column_types': '["Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric"]', 'column_names': '["cut","color","clarity","carat","depth_p","table_p","price"]', 'source_frames': '["upload_aaba0099ca02d395d0dcc4d4a36849c6"]'}


In [1]:
train

NameError: name 'train' is not defined

In [8]:
valid

cut,color,clarity,carat,depth_p,table_p,price
4,19,6,1.01,62.7,58,8.754
2,19,5,0.6,64.3,57,7.492
3,20,3,1,62.8,57,8.314
4,20,4,0.7,62.1,59,7.694
2,16,6,0.71,63.3,55,7.54
3,19,4,0.9,63,57,8.211
3,18,3,2.01,63.4,57,9.687
5,19,5,0.56,61.9,55,7.365
2,18,4,1.5,64.9,57,9.001
4,22,4,0.3,63,58,6.564


In [9]:
# Build Model   
dia_drf = H2ORandomForestEstimator(ntrees=10000,max_depth=2000,nfolds=5)

In [10]:
#Train
dia_drf.train(x=predictors, y=response, training_frame=train)


drf Model Build progress: | (failed)


OSError: Job with key $03017f00000134d4ffffffff$_a24af904663bf87c793748d41d3e496e failed with an exception: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: DRF_model_python_1620499930463_3_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (171.1 KB per tree x 10000 > 117.3 MB) - try decreasing ntrees and/or max_depth or increasing min_rows!

stacktrace: 
water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: DRF_model_python_1620499930463_3_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (171.1 KB per tree x 10000 > 117.3 MB) - try decreasing ntrees and/or max_depth or increasing min_rows!

	at water.exceptions.H2OModelBuilderIllegalArgumentException.makeFromBuilder(H2OModelBuilderIllegalArgumentException.java:19)
	at hex.tree.SharedTree.doScoringAndSaveModel(SharedTree.java:721)
	at hex.tree.SharedTree$Driver.s

In [31]:
performance = dia_drf.model_performance(test_data=valid)

print(performance)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
x_test=pd.read_csv('./data/test.csv')
x_test=limpia_catergoricas(x_test)
x_test=h2o.H2OFrame(x_test)


df_sub=pd.DataFrame(regr.predict(x_test))
df_sub.reset_index(inplace=True)
df_sub.columns=['id','price']
# Generate predictions on a validation set (if necessary):
pred = dia_drf.predict(valid)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 20000, num = 100)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 101, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

random_grid = {'n_estimators': [1000],
               'max_features': [6],
               'max_depth': [100],
               'min_samples_split': [10],
               'min_samples_leaf': [3],
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
rf=RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 1, cv = 3, verbose=3, n_jobs = -1, )

x=df.drop('price',axis=1)
y=df.price
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
rf_random.fit(x_train, y_train)

duration=1
freq=440
os.system('play -nq -t  alsa synth {} sine {}'.format(duration,freq))
os.system('play -nq -t  alsa synth {} sine {}'.format(duration,2*freq))
os.system('play -nq -t  alsa synth {} sine {}'.format(duration,freq))

mse(y_test,rf_random.predict(x_test))


In [6]:
x=df.drop('price',axis=1)
y=df.price
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=23)

regr = RandomForestRegressor(n_estimators=2000, bootstrap=True,max_features=5,min_samples_split=2, min_samples_leaf=1, warm_start=True)
regr.fit(x_train, y_train)
mse(y_test,regr.predict(x_test))



0.008498457654443766

In [7]:
x_test=pd.read_csv('./data/test.csv')
x_test=limpia_catergoricas(x_test)
x_test=h2o.H2OFrame(x_test)
=regr.predict())
# Eval performance:
y_tests = dia_drf.model_performance(x_test)
df_sub.reset_index(inplace=True)
df_sub.columns=['id','price']


In [27]:
df_size = df[df.carat - np.floor(df.carat) == 0]
df_off = df[df.carat - np.floor(df.carat) != 0]


In [33]:
df_sub.to_csv('./out/sub_1stramforestn_estimators=2000max_features=4min_samples_split=20warm_start=True.csv',index=False)

In [9]:
from hpsklearn import HyperoptEstimator, sgd

# HyperOpt
from hyperopt import hp

# Sklearn stuff
from sklearn.model_selection import train_test_split
import numpy as np

# Load and split dataset
x=df.drop('price',axis=1)
y=df.price
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=23)

# Penalizer for SGD
penalty = 'l2'

# Search space of "loss functions"
# Different loss variations are defined, each one
# with an specific probability to be selected
# In this case, "hinge" is prioritized with respect the others
loss = hp.pchoice('loss', [(0.50, 'hinge'), 
                           (0.25, 'log'), 
                           (0.25, 'huber')])

# Search space for alpha value is definied
# with a log-uniform function
print(max(y)/5)

alpha = hp.loguniform('alpha', low=min(y)/5, high=max(5)*5)

# Init Estimator
# The classifier is passed with its respective hyperparameters
model = HyperoptEstimator(classifier=sgd('my_sgd', 
                                         penalty=penalty, 
                                         loss=loss, 
                                         alpha=alpha), 
                          max_evals=10, 
                          trial_timeout=30)

# Training
model.fit(x_train, y_train)

# Results
print(f"Training score: {model.score(x_train, y_train)}")
print(f"Test score: {model.socre(x_test, y_test)}")

# Optimal configuration
print(f"Optimal configuration: {model.best_model()}")

1.9686


TypeError: 'int' object is not iterable

In [ ]:


X_test=pd.read_csv('./data/test.csv')

X_test=limpia_catergoricas(X_test)

X_test.columns=['id', 'carat', 'cut', 'color', 'clarity', 'depth_p', 'table_p', 'x', 'y', 'z']

X_test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25)

In [ ]:
lr=LinearRegression()
lr.fit(X_train,y_train)


In [ ]:
lr=LinearRegression()
lr.fit(X,y)


In [ ]:
X

In [30]:
X_test=pd.read_csv('./data/test.csv')

In [31]:
X_test=limpia_catergoricas(X_test)

In [32]:
df_sub=pd.DataFrame(lr.predict(X_test))

NameError: name 'lr' is not defined

In [ ]:
df_sub.reset_index(inplace=True)


In [ ]:
df_sub

In [ ]:
df_sub.columns=['id','price']

In [ ]:
df_sub.to_csv('./out/sub1.csv',index=False)